In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf
from pyspark.sql.types import IntegerType

from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, LinearSVC, GBTClassifier

from pyspark.sql import Window
from pyspark.sql.functions import sum as Fsum

import re

VBox()

VBox()

Starting Spark application
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1546726294751_0006,pyspark,idle,Link,Link,✔


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1546726294751_0006,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
SparkSession available as 'spark'.


In [2]:
# Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

# Read in full sparkify dataset
event_data = "s3n://dsnd-sparkify/sparkify_event_data.json"
df = spark.read.json(event_data)
df.head()

VBox()

VBox()

Row(artist=u'Popol Vuh', auth=u'Logged In', firstName=u'Shlok', gender=u'M', itemInSession=278, lastName=u'Johnson', length=524.32934, level=u'paid', location=u'Dallas-Fort Worth-Arlington, TX', method=u'PUT', page=u'NextSong', registration=1533734541000, sessionId=22683, song=u'Ich mache einen Spiegel - Dream Part 4', status=200, ts=1538352001000, userAgent=u'"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId=u'1749042')Row(artist=u'Popol Vuh', auth=u'Logged In', firstName=u'Shlok', gender=u'M', itemInSession=278, lastName=u'Johnson', length=524.32934, level=u'paid', location=u'Dallas-Fort Worth-Arlington, TX', method=u'PUT', page=u'NextSong', registration=1533734541000, sessionId=22683, song=u'Ich mache einen Spiegel - Dream Part 4', status=200, ts=1538352001000, userAgent=u'"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId=u'1749042')

In [3]:
# remove na
df = df.filter(df.userId != "")

VBox()

VBox()

In [4]:
# Create days Df 
minDf = df.select(["userId", "ts"]).groupby("userId").min("ts")
maxDf = df.select(["userId", "ts"]).groupby("userId").max("ts")
daysCount = minDf.join(maxDf, on="userId").withColumn("days", (col("max(ts)")-col("min(ts)")) / (1000*60*60*24)).select(["userId", "days"])

VBox()

VBox()

In [5]:
# Create sessions Df 
sessionCount = df.select(["userId", "sessionId"]).drop_duplicates()\
.groupby("userId").count().withColumnRenamed("count", "sessions")

VBox()

VBox()

In [6]:
# Extract initial feature dataframe
featureDf = df.groupby(["userId"]).pivot("page").count().na.fill(0)

VBox()

VBox()

In [7]:
# Join in session count
featureDf = featureDf.join(sessionCount,on="userId")

VBox()

VBox()

In [8]:
# Join in days count
featureDf = featureDf.join(daysCount,on="userId")

VBox()

VBox()

In [9]:
# Join in churn as label
featureDf = featureDf.withColumn("label", featureDf["Cancellation Confirmation"])

VBox()

VBox()

# Function Definition

In [10]:
f1Evaluator = MulticlassClassificationEvaluator(metricName="f1")

def EvaluatePredictions(predictions):
    print("F1: "+ str(f1Evaluator.evaluate(predictions)))
    
    print("Confusion Matrix:")
    predictions.groupby("label").pivot("prediction").count().show()

VBox()

VBox()

# Modeling

In [11]:
# Split data into train, test and validation set
training, test, validation = featureDf.randomSplit([0.8, 0.1, 0.1])

VBox()

VBox()

In [12]:
# Construct MinMaxScaler and Classifier
a = VectorAssembler(inputCols=["sessions", "days", "About", "Add Friend", "Add to Playlist", "Help", "Home", "NextSong", "Settings", "Thumbs Up", "Upgrade"], outputCol="Features")

s = MinMaxScaler(inputCol="Features", outputCol="ScaledFeatures")

rf = GBTClassifier(featuresCol="ScaledFeatures")
#rf = RandomForestClassifier(featuresCol="ScaledFeatures")
#rf = LinearSVC(featuresCol="ScaledFeatures") # maxIter=5, regParam=0.01

# Setup Pipeline
pipeline = Pipeline(stages=[a, s, rf])

VBox()

VBox()

In [13]:
# Train Pipeline on test data
pipeModel = pipeline.fit(training)

VBox()

VBox()

In [14]:
vPredictions = pipeModel.transform(validation)

VBox()

VBox()

In [15]:
EvaluatePredictions(vPredictions)

VBox()

VBox()

F1: 0.890975204126
Confusion Matrix:
+-----+----+---+
|label| 0.0|1.0|
+-----+----+---+
|    0|1721| 71|
|    1| 164|294|
+-----+----+---+F1: 0.890975204126
Confusion Matrix:
+-----+----+---+
|label| 0.0|1.0|
+-----+----+---+
|    0|1721| 71|
|    1| 164|294|
+-----+----+---+